# Aprendizado por instância

Robson Mesquita Gomes <robson.mesquita56@gmail.com>

###### Importações Iniciais

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import io
import requests

##### Preparação para captura de dados

In [41]:
def getDataByURL(url):
    
    data = requests.get(url).content
    
    return io.StringIO(data.decode('utf-8'))

### Dados Titanic

O dataset Titanic contém 5 atributos (Nome, Classe, Idade, Sexo, Sobreviveu) com 1313 instâncias, cada uma representando um dos passageiros ou tripulantes do RMS Titanic que naufragou no Oceano Atlântico em 15 de abril de 1912. 

Com esses dados queremos inferir se uma determinada pessoa sobreviveu ou não ao naufrágio ( `y = {Sobreviveu}` ), usando como base os atributos descritivos `× = {Classe, Idade, Sexo]`. Logo o problema em questão é de _Classificação_, e o atributo alvo _Sobreviveu_ contém apenas duas classes: `1`, se a pessoa sobreviveu e `0`, se a pessoa não sobreviveu.

In [42]:
df = pd.read_csv(getDataByURL('https://query.data.world/s/c2ixg24yogu6pgudqodbhn5ynrvdjr'))

df.head()

,Nome,Classe,Idade,Sexo,Sobreviveu
0,"Allen, Miss Elisabeth Walton",1.0,29.00,F,1
1,"Allison, Miss Helen Loraine",1.0,2.00,F,0
2,"Allison, Mr Hudson Joshua Creighton",1.0,30.00,M,0
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1.0,25.00,F,0
4,"Allison, Master Hudson Trevor",1.0,0.92,M,1


In [43]:
len(df)

1313

#### Criando o atributo 'FaixaEtaria'

A fim de simplificar nossas nálises, vamos discretizar o atributo numérico `idade`, transformando-o no atribuo categórico `FaixaEtaria`, contendo as seguintes classes conforme o intervalo de idades:  

- 0 a 10: Criança
- 11 a 18: Jovem
- 19 a 50: Adulto
- +50: Idoso
- Não conhecida

In [44]:
df['Idade'].unique()

array([29.  ,  2.  , 30.  , 25.  ,  0.92, 47.  , 63.  , 39.  , 58.  ,
       71.  , 19.  ,   nan, 50.  , 24.  , 36.  , 37.  , 26.  , 28.  ,
       45.  , 22.  , 41.  , 48.  , 44.  , 59.  , 60.  , 53.  , 33.  ,
       14.  , 11.  , 49.  , 46.  , 27.  , 31.  , 64.  , 55.  , 70.  ,
       69.  , 38.  , 17.  ,  4.  , 23.  , 35.  , 54.  , 21.  , 52.  ,
       16.  , 51.  , 42.  , 40.  , 15.  , 65.  , 18.  , 56.  , 43.  ,
       61.  , 13.  , 34.  ,  6.  , 57.  , 32.  , 62.  , 67.  , 20.  ,
        1.  , 12.  ,  0.83,  8.  ,  7.  ,  3.  ,  0.8 ,  9.  ,  5.  ,
        0.33,  0.17, 10.  ,  1.5 ])

In [45]:
# Transforma idade em faixa etária

def faixaEtaria(idade):
    
    if idade == np.nan:
        return 'NaoConhecida'
    
    if 0.0 <= idade <= 10.0:
        return 'Crianca'
    
    if 11.0 <= idade <= 18.0:
        return 'Jovem'
    
    if 19.0 <= idade <= 50.0:
        return 'Adulto'
    
    if idade > 50.0:
        return 'Idoso'

faixaEtaria(51) # <-- teste

'Idoso'

Criando o atributo `FaixaEtaria` e associando à ele uma lista com os valores de faixa etária para cada valor de idade

In [46]:
df['FaixaEtaria'] = [faixaEtaria(idade) for idade in df['Idade'].values]

df.head()

,Nome,Classe,Idade,Sexo,Sobreviveu,FaixaEtaria
0,"Allen, Miss Elisabeth Walton",1.0,29.00,F,1,Adulto
1,"Allison, Miss Helen Loraine",1.0,2.00,F,0,Crianca
2,"Allison, Mr Hudson Joshua Creighton",1.0,30.00,M,0,Adulto
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1.0,25.00,F,0,Adulto
4,"Allison, Master Hudson Trevor",1.0,0.92,M,1,Crianca


#### Seleção de atributos

Excluiremos do DataFrame os atributos `Nome` e `Idade`, pois não serão úteis em nossas nálises daqui para a frente, mantendo os atributos `Classe`, `FaixaEtaria` e `Sobreviveu`.

In [47]:
df = df[['Classe', 'FaixaEtaria', 'Sexo', 'Sobreviveu']]

df.head()

,Classe,FaixaEtaria,Sexo,Sobreviveu
0,1.0,Adulto,F,1
1,1.0,Crianca,F,0
2,1.0,Adulto,M,0
3,1.0,Adulto,F,0
4,1.0,Crianca,M,1


In [48]:
df.loc[3]

Classe            1.0
FaixaEtaria    Adulto
Sexo                F
Sobreviveu          0
Name: 3, dtype: object

#### Amostrando os conjuntos de Treino e de Teste

A fim de investigar o poder de generalização dos métodos estudados, vamos separar os dados em dois conjuntos:

- **Conjunto de Treino**: Contém 70% dos dados originais e será utilizado no processo de inferência
- **Conjunto de Teste**: Contém 30% dos dados originais e será utilizado

In [49]:
total = len(df.index)

treino = df.iloc[: int(total * .7)]
teste = df.iloc[ int(total * .7) :]

In [50]:
treino.index

RangeIndex(start=0, stop=919, step=1)

### Matriz de Confusão e Acurácia

A matriz de confusão contém os valores:

- $TP$ - Verdadeiros Positivos
- $FP$ - Falsos Positivos
- $TN$ - Verdadeiros Negativos
- $FN$ - Falsos Negativos

Com esses valores definimos a métrica de Acurácia $A_c \in [0,1]$, tal que:  

$$ A_c = {{TP + TN} \over {TP + FP + TN + FN}} $$

In [51]:
def matrizConfusao(esperados, estimados):
    
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for par in zip(esperados, estimados):
        
        if par[0] and par[1]:
            tp += 1
        
        if par[0] and not par[1]:
            fn += 1
        
        if not par[0] and not par[1]:
            tn += 1
        
        if not par[0] and par[1]:
            fp += 1

    return np.array([tp, fp, tn, fn])

In [52]:
def acuracia(esperados, estimados):
    n = len(esperados)
    
    tp, fp, tn, fn = matrizConfusao(esperados, estimados)
    
    return (tp + tn)/n

In [53]:
# teste

esperados = [0, 0, 0, 0]
estimados = [0, 1, 0, 1]

acuracia(esperados, estimados)

0.5

# kNN

K-Vizinhos Mais Próximos

## Função de distância

Uma função de distância $d : \mathbb{R}^n, \mathbb{R}^n \rightarrow \mathbb{R}^+$ é uma função que calcula a similaridade de dois vetores $\alpha \in \mathbb{R}^n$ e $\beta \in \mathbb{R}^n$ através de sua distância no espaço $\mathbb{R}^n$.

No caso de instâncias em que os valores não são reais, mas categóricos, uma função de distância comum é a [Distância de Hamming](https://pt.wikipedia.org/wiki/Distância_de_Hamming), que pode ser definida como:

$$d(\alpha, \beta) = \sum_{i=0}^{n} \mathbb{I}(\alpha[i] \ne \beta[i])$$

Onde $\mathbb{I}$(expressão)$\{$retorna $1$ se expressão for verdadeira e $0$ se falsa$\}$ é a função indicadora.

In [54]:
def distanciaHamming(a,b):
    return sum( [a[key] != b[key] for key in a.keys()] )

In [55]:
#teste

a = {'a':1, 'b': 0, 'c':1}
b = {'a':1, 'b': 1, 'c':1}

distanciaHamming(a,b)

1

Na nossa implementação a Distância de Hamming é normalizado no intervalo $[0,1]$

In [56]:
def distanciaHammingNormalizada(a, b):
    return distanciaHamming(a,b)/len(a.keys())

In [57]:
#teste

a = {'a':1, 'b': 0, 'c':1}
b = {'a':1, 'b': 1, 'c':1}

distanciaHammingNormalizada(a,b)

0.3333333333333333

Uma aplicação possível é:

In [58]:
print("Passageiro 1: \n{}\n\n{}\n\n".format( "="*13, df.iloc[1]))

print("Passageiro 2: \n{}\n\n{}\n\n".format( "="*13, df.iloc[2]))

print("Distância: {}".format(distanciaHammingNormalizada(df.iloc[1], df.iloc[2])))

Passageiro 1: 

Classe             1.0
FaixaEtaria    Crianca
Sexo                 F
Sobreviveu           0
Name: 1, dtype: object


Passageiro 2: 

Classe            1.0
FaixaEtaria    Adulto
Sexo                M
Sobreviveu          0
Name: 2, dtype: object


Distância: 0.5


## Função de Mesclagem

In [59]:
from collections import Counter

In [60]:
def maisFrequente(array):
    frequencias = Counter(array)
    return max(frequencias, key=frequencias.get)

In [61]:
maisFrequente([1,1,0,0,1,0,0]) # teste

0

### Calculando e ordenando distâncias

In [62]:
x = {'Sexo': 'M', 'FaixaEtaria': 'Adulto', 'Classe': 3.0}

##### Calculando distâncias

In [63]:
distancias = { indice: distanciaHammingNormalizada(x, treino.loc[indice]) for indice in treino.index}

##### Ordenando Índices por Distâncias

In [64]:
indices = sorted(distancias, key=distancias.get)

### Selecionando os vizinhos mais próximos e mesclando os valores

In [65]:
vizinhos = treino.loc[indices[ :3]]

vizinhos

,Classe,FaixaEtaria,Sexo,Sobreviveu
602,3.0,Adulto,M,0
607,3.0,Adulto,M,1
609,3.0,Adulto,M,1


In [66]:
y = maisFrequente(vizinhos['Sobreviveu'].values)

y

1

## Algoritmo kNN

In [67]:
def kNN(dados, k, x, y, funcaoDistancia, funcaoMesclagem):
    
    distancias = { indice: funcaoDistancia(x, dados.iloc[indice]) for indice in dados.index}
    
    indices = sorted(distancias, key=distancias.get)
    
    vizinhos = dados.loc[indices[ :k]]
    
    yEstimado = funcaoMesclagem(vizinhos[y].values)
    
    return yEstimado

In [68]:
passageiro = {'Sexo': 'F', 'FaixaEtaria': 'Adulto', 'Classe': 1.0}

kNN(
    treino,
    2,
    passageiro,
    'Sobreviveu',
    distanciaHammingNormalizada,
    maisFrequente
)

1

## Teste do modelo

In [69]:
x = ['Sexo', 'Classe', 'FaixaEtaria']
y = 'Sobreviveu'

esperados = [] # valores existentes no conjunto de testes
estimados = [] # gerados pelo kNN

for indice in teste.index:
    instancia = teste.loc[indice]
    
    esperados.append(instancia[y])
    
    consulta = {chave: instancia[chave] for chave in x}
    
    y_est = kNN(treino, 2, consulta, y, distanciaHammingNormalizada, maisFrequente)
    
    estimados.append(y_est)

acuracia(esperados, estimados)

0.7233502538071066

## Naïve Bayes

##### Regra de Bayes

$$P(y|x) = {{P(x|y) \cdot P(y)} \over {P(x)}}$$

_onde:_

- $x$ _são os valores dos atributos descritivos (ou a evidência)_
- $y$ _é o valor do atributo alvo (ou hipótese)_
- $P(x|y)$ _é a probabilidade à posteriore_
- $P(x|y)$ _é a verossimilhança_
- $P(y)$ _é a probabilidade à priori_
- $P(x)$ _é a probabilidade da evidência_

#### Probabilidade da evidência - $P(x)$

In [78]:
# Tamanho total do conjunto de dados
n = len(treino.index)

# Atributos descritivos
x = "Sexo == 'M' & Classe == 3"

# Filtra o conjunto de dados a partir dos atributos descritivos
xDados = treino.query(x)

# Tamanho total do filtro
xn = len(xDados.index)

# Probabilidade Empírica P(x)
px = xn/n

px

0.24156692056583243

#### Probabilidade à priori - $P(y)$

In [79]:
# Número de classes do atributo alvo
classes = treino['Sobreviveu'].unique()

classes

array([1, 0])

In [80]:
# Dicionário que conterá as probabilidades P(y) para cada classe de y
py = {}

for classe in classes:
    
    # Filtra o conjunto de dados a partir dos atributos alvo
    yDados = treino.query("Sobreviveu == {}".format(classe))
    
    # Tamanho do filtro
    yn = len(yDados.index)
    
    # Probabilidade à priori da classe y (P(y))
    py[classe] = yn/n
    

py
    

{1: 0.42219804134929273, 0: 0.5778019586507073}

#### Verossimilhança - $P(x|y) = {P(x,y) \over P(y)}$

In [81]:
# Dicionário que conterá as probabilidades P(x|y) para cada classe de y
pxy = {}

for classe in classes:
    
    # Filtra o conjunto de dados a partir dos atributos descritivos e dos atributos alvo
    xyDados = treino.query("{} & Sobreviveu == {}".format(x, classe))
    
    # Tamanho do filtro
    xyn = len(xyDados.index)
    
    # Verossimilhança dos atributos descritivos em relação aos atributos alvo
    pxy[classe] = (xyn/n)/py[classe]

pxy

{1: 0.07731958762886598, 0: 0.3615819209039548}

#### Probabilidade à Posteriori - $P(y|x) = {{P(x|y) \cdot P(y)} \over {P(x)}}$

In [82]:
# Dicionário que conterá as probabilidades P(y|x) para cada classe de y
pyx = {}

for classe in classes:
    # Regra de Bayes P(y|x) = (P(x|y)*P(y))/P(x)
    pyx[classe] = (pxy[classe]*py[classe])/px

pyx

{1: 0.13513513513513514, 0: 0.8648648648648649}

### Máximo à Posteriori - $MAP$

A classificação é decidida pela classe $y_i$ que maximizar a probabilidade à posteriori $P(y_i|x)$  

$$y_{MAP} = \arg\max_{y_i}P(y_i|x) $$

In [83]:
max(pyx, key=pyx.get)

0

## Algorítmo Naïve Bayes

In [86]:
def naiveBayes(dados, x, y):
    n = dados.size
    
    # Probabilidade da evidência P(x)
    xDados = dados.query(x)
    xn = xDados.size
    px = xn/n
    
    # Probabilidade à priori P(y) por classe
    classes = dados[y].unique()
    py = {}
    
    for classe in classes:
    
        yDados = dados.query("{} == {}".format(y, classe))
        yn = yDados.size
        
        py[classe] = yn/n
    
    # Verossimilhança P(x|y) por classe
    pxy = {}
    
    for classe in classes:
        xyDados = dados.query("{} & {} == {}".format(x, y, classe))
        xyn = xyDados.size
        
        pxy[classe] = (xyn/n)/py[classe]
    
    # Probabilidade à posteriori - P(y|x) por classe
    pyx = {}
    
    for classe in classes:
        # Teorema de Bayes P(y|x) = (P(x|y)*P(y))/P(x)
        pyx[classe] = (pxy[classe]*py[classe])/px
    
    return max(pyx, key=pyx.get)

In [91]:
naiveBayes(treino, "Sexo == 'F' & Classe == 1", 'Sobreviveu')

1

In [89]:
naiveBayes(treino, "FaixaEtaria == 'Adulto' & Classe == 3", 'Sobreviveu')

0

### Teste do modelo

In [97]:
x = ['Sexo', 'Classe', 'FaixaEtaria']
y = 'Sobreviveu'

esperados = []
estimados = []

for indice in teste.index:
    instancia = teste.loc[indice]
    
    esperados.append(instancia[y])
    
    tmp = [ "{} == '{}'".format(chave, instancia[chave]) for chave in x ]
    consulta = ' & '.join(tmp)
    
    y_est = naiveBayes(teste, consulta, y)
    
    estimados.append(y_est)

acuracia(esperados, estimados)

/var/folders/2y/q957rqmj3c1ckz7t4qg_1cdr0000gn/T/ipykernel_32293/1784555562.py:34: RuntimeWarning: invalid value encountered in scalar divide
  pyx[classe] = (pxy[classe]*py[classe])/px


0.8426395939086294